In [43]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', None)

In [44]:
headers = {
    'authority': 'www.zillow.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'cookie': f"zguid=24|%24b10a5e0a-e13d-4c5c-960b-5b31dcca0531; _ga=GA1.2.1375033491.1654444401; zjs_user_id=null; zg_anonymous_id=%22ae5ecbbb-bd46-4807-9e44-9f2acf1a3a14%22; zjs_anonymous_id=%22b10a5e0a-e13d-4c5c-960b-5b31dcca0531%22; _pxvid=a042ad1b-e4e7-11ec-a384-4b414e514e43; _gcl_au=1.1.1281869689.1654444402; __pdst=c64c2d41b05747c28b2da93bd6b2a4e8; _fbp=fb.1.1654444401994.1357991477; _cs_c=0; _pin_unauth=dWlkPU1XTTNOMkkwTldJdE0ySmtOQzAwTm1FNUxUa3hPRGd0T0RRM1lUZGpNMlExTlRNNQ; __gads=ID=f7f6bac03ea6a4f0:T=1654444410:S=ALNI_Many6kPLwj0PALCdt_4FbexVoqIYQ; zgsession=1|43394f21-f325-4669-9c21-9e3e911f7876; pxcts=cbd8d929-f55d-11ec-a4b0-4353574e6255; DoubleClickSession=true; _cs_id=e28ee22c-d70c-a12e-b9c0-389a3739d756.1654444402.2.1656254374.1656254374.1.1688608402224; g_state={\"i_p\":1656956633369,\"i_l\":1}; utag_main=v_id:01813492256500156a841e2694d805079001707100e50{_sn:3$_se:1$_ss:1$_st:1656951234269$dc_visit:2$ses_id:1656949434269%3Bexp-session$_pn:1%3Bexp-session;} _hp2_id.1215457233=%7B%22userId%22%3A%221466810969009166%22%2C%22pageviewId%22%3A%224827358997361410%22%2C%22sessionId%22%3A%225371662746300312%22%2C%22identity%22%3A%22b10a5e0ae13d4c5c960b5b31dcca0531%22%2C%22trackerVersion%22%3A%224.0%22%2C%22identityField%22%3Anull%2C%22isIdentified%22%3A1%7D; _gid=GA1.2.1363446346.1657762596; KruxPixel=true; JSESSIONID=8744FAF401E29A22050D960448582B32; __gpi=UID=0000062a50801980:T=1654444410:RT=1657762648:S=ALNI_MYDXlAakJ16IyQbgNOz0a4Bnwl3MA; KruxAddition=true; _clck=1e9fp7s|1|f35|0; _clsk=ubs3ud|1657763007139|8|0|k.clarity.ms/collect; _gat=1; _pxff_bsco=1; _uetsid=84c39430031511eda89ad7762e8966db; _uetvid=b8f9d2606e7a11ec9fd78b4c24e0cd9c; AWSALB=a1g4dG1XqWqWlJMyzX7sQYA1p3TqhS/B2NWBCQzjKQriKybWRHlEi0++J8iATjkY8P1w4/lXUx464xnSxkcwyu713ay9njfUkazWW03sBl7NEaK1eJkhkexJtTSk; AWSALBCORS=a1g4dG1XqWqWlJMyzX7sQYA1p3TqhS/B2NWBCQzjKQriKybWRHlEi0++J8iATjkY8P1w4/lXUx464xnSxkcwyu713ay9njfUkazWW03sBl7NEaK1eJkhkexJtTSk; search=6|1660355098867%7Cregion%3Doklahoma-city-ok%26rb%3DOklahoma-City%252C-OK%26rect%3D35.673098%252C-97.203126%252C35.290544%252C-97.706877%26disp%3Dmap%26mdm%3Dauto%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%09%0933225%09%09%09%09%09%09; _px3=400e6bb6c89676b9320d9ee16242c1795af5059532d7a1caae4f1cabf01ae79a:q07m+CbJg1cw4uSaKYZiY86pCfrrkSHB3iC+PqIBs6uCbiEZOBn/okDcZFURm1QnpEvql3Sg3MVKHMWLghBZmA==:1000:xIM+klCYyqQLXLssjHpP+5gjXuBcPCzQ4t7mTWqJ4MytyPbgWfFvJShgMGBtIcv75EpR5alQ9gWkMylNIrNgBg8plglsaYtjRK5JbkLoHIPQazfMOaLLmhfBryg0/FzjqcpQ44hN3Ho/RlXUa1BUj2z3sbcJ6xm+t90xYfNZV8yKNwBFOAR2ZFrCl3Tr/XEce2LqyRMjBPrhHGR5qHt4hw==",
    'referer': 'https://www.zillow.com/oklahoma-city-ok/',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}


In [45]:
with requests.session() as s:
    city = 'oklahoma-city-ok/'
    page = 1
    end_page = 20
    url = ''
    url_list = []
    
    while page <= end_page:
        url = 'https://www.zillow.com/' +city+ f'{page}_p/'
        url_list.append(url)
        page +=1
        
    request = ''
    request_list = []
    
    for url in url_list:
        request = s.get(url, headers=headers)
        request_list.append(request)
        
soup = ''
soup_list = []

for request in request_list:
    soup = BeautifulSoup(request.content, 'html.parser')
    soup_list.append(soup)

In [46]:
soup_list[0]

<!DOCTYPE html>
<html class="no-js zsg-theme-modernized null" itemscope="" itemtype="http://schema.org/Organization" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#">
<head>
<title>Oklahoma City Real Estate - Oklahoma City OK Homes For Sale | Zillow</title><meta content="Zillow has 1,335 homes for sale in Oklahoma City OK. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/><meta content="Zillow, Inc." name="author"/><meta content="Copyright (c) 2006-2022 Zillow, Inc." name="Copyright"/><script>
        if (typeof Object.assign === 'function') {
            window.appInfo = Object.assign(
                typeof window.appInfo === 'object' ? window.appInfo : {},
                {"@zillow/page-frame-content":"4da388b5"}
            );
        }
    </script><script>
        window.CLIENT_PROFILER_

In [47]:
df_list = []
for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        address = soup.find_all('address')
        
        df['address'] = address
    df_list.append(df)

In [48]:
df_list[0]

,address
0,"[7540 NW 11th Street, Oklahoma City, OK 73127]"
1,"[2134 NW 15th Street, Oklahoma City, OK 73107]"
2,"[6721 Cherry Place, Oklahoma City, OK 73127]"
3,"[2601 NW 112th Street, Oklahoma City, OK 73120]"
4,"[3809 NW 34th Street, Oklahoma City, OK 73112]"
5,"[2808 SW 115th Street, Oklahoma City, OK 73170]"
6,"[8802 S Sunnylane Road, Oklahoma City, OK 73135]"
7,"[4809 NW 29th Street, Oklahoma City, OK 73127]"
8,"[715 NW 89th Street, Oklahoma City, OK 73114]"


In [ ]:
len(df_list[0])